### Generics
Generics enables Type to be parameters. In doing so it provides the following benefits:
- Code re-use

In [ ]:
public int add(int a, int b){
    return a + b;
}

public float add(float a, float b){
    return a + b;
}

    Can be written as:

In [ ]:
public T add(T a, T b){ // But we would need to limit
    return a + b;       // types T can be
}

- Elimination of casts

In [ ]:
List oldList = new ArrayList();
oldList.add("some text");
String str = (String) oldList.get(0);

// Can be written as
List<String> newList = new ArrayList<>();
newList.add("some text");
String str = newList.get(0);

    This introduces type safety, saving us from class cast exceptions.

### Generic Class
We define a generic class by specifying a type parameter inside <>. The type parameter need not be single letter, it can be a full word.

In [ ]:
class Box<T> { // Specifying T here means this type parameter is now available
               // throughout the class
    private T t;

    public Box(T t){
        this.t = t;
    }

    public T get() {
        return t;
    }

    public void set(T t) {
        this.t = t;
    }
}

Box<Number> numBox = new Box<>(5); // Type inference

Multiple type parameters can also be specified 

In [ ]:
class Pair<K, V> { // Different type parameters are just separated by a comma
    private K key;
    private V value;

    public void set(K key, V value) {
        this.key = key;
        this.value = value;
    }

    public V getValue() {
        return this.value;
    }

    public V getKey() {
        return this.value;
    }
}

We can also have a generic method inside a non-generic class

In [ ]:
class Util {
    // Notice <T>, we have to define type parameter first
    // since class is non-generic
    public static <T> void fromArrayToCollection(T[] a, Collection<T> c) { // Type parameter T is limited
                                                                           // to this method
        for (T o : a) {
            c.add(o);
        }
    }
}

**Raw type:** is the name of a generic class or interface without any type arguments. For example:

In [ ]:
Box rawBox = new Box();

// If we assign a raw type to a parameterized type, you get a warning
Box<String> stringBox = new Box("Hello");

// Since the reference is generic, we can only call
stringBox.set("Another value");
stringBox.set(55);    // Error

**Bounded Type Parameter:** lets us restrict the types that can be used as type arguments.

In [ ]:
class Complex <T extends Number> {
    public T real;
    public T imaginary;
}

In [ ]:
// Multiple bounds, useful if T implements multiple interfaces
class AnimalCollection <T extends Vertebrate & Warmblooded> {
    // ...
}

class Mammal extends Vertebrate implements Warmblooded { // Mammal can be used as Type argument
    // ...
}

class Reptile extends Vertebrate{ // Reptiles can't be used as Type argument
    // ...
}

### Generics And Inheritance
Regarding inheritance and generic type we should note that (using above example) `Box<Integer>` is not subtype of `Box<Number>` even though `Integer` is subtype of `Number`. So:

In [ ]:
public void compute(Box<Number> input){
    // ...
}

// Below method call is invalid
compute(new Box<Integer>()); // Error

// Passing a raw type is legal, but we get warning
compute(new Box());

If we want to extend a generic class, the child class must include the type specified by the parent class.

In [ ]:
// Valid
class SpecialList<T> extends ArrayList<T> {

}

// Valid, child class introducing its own parameter
class SpecialList<T, U> extends ArrayList<T> {

}

// Valid, generic class can extend non-generic class
class SpecialList<T> extends ArrayList {

}

// Error, child class must have all type parameters
// of the parent class
class SpecialList<T> extends ArrayList<U> {

}

// Error, child class should also have the interface's type
// parameter. Type parameters of child class should be union 
// of type parameters of generic class and generic interface
class SpecialList<T> extends ArrayList<T> implements Comparable<U>{
    
}

// Error, non-generic class can’t extend generic class
class SpecialList extends ArrayList<T>{
    
}

// But this is fine, since we are using a pre-defined class
class SpecialStringList extends ArrayList<String>{
    
}

// String is a type parameter, not String class
class SpecialStringList<String> extends ArrayList<String>{
    
}
SpecialStringList<Integer> integerList = new SpecialStringList<>();

When we extend a generic class with bounded type parameter, type parameter must be replaced by either upper bound or it’s sub classes.

In [ ]:
class Parent<T extends Number> {

}

// T can be any type
class Child<T> extends Parent<Integer> {

}

### Wildcards
There may be times when we want to restrict the kinds of types that are allowed to be passed to a type parameter. For example, say you want to write a method that works on `List<Integer>`, `List<Double>`, and `List<Number>`. We use *upper bounded wildcard*:

In [ ]:
public void process(List<? extends Number> src, List<? extends Number> dest) {
    // src and destination can be List<Integer> and List<Double> respectively
}

Note that this is different from:

In [ ]:
public <T extends Number> void process(List<T> src, List<T> dest) {
    // src and destination both need to be the same type
}

But, the following code has the same functionality as the wildcard version

In [ ]:
public <T extends Number, U extends Number> void process(List<T> src, List<U> dest) {
    // src and destination can be List<Integer> and List<Double> respectively
}

The wildcard version however is more concise and readable. Another advantage comes with *lower bounded wildcard*.

In [ ]:
public void process(List<? super Number> src) {
    // Type parameter of List must atleast be Number
}

There is a simple guideline when to use upper bounded wildcard and when to use lower bounded wildcard
- parameter which acts as data source should use upper bound
- parameter which acts as data consumer should use lower bound

PECS : Producer Extends Consumer Super

The class hierarchy is illustrated in the image below:  
![Generic Suubtypes](https://docs.oracle.com/javase/tutorial/figures/java/generics-wildcardSubtyping.gif)